# XGBoost Regression with TensorFlow Pooling and Loss
## Tutorial on early stopping
This tutorial demonstrates the use of `xgb_tf_metric` decorator for early stopping. For a more comprehensive tutorial on how to use `tf2xgb` library, please refer to [this](https://github.com/petrkaderabek/tf2xgb/blob/main/example.ipynb) example.

In [1]:
import numpy as np
import xgboost as xgb
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt

from tf2xgb import get_ragged_nested_index_lists, gen_random_dataset, xgb_tf_loss, xgb_tf_metric
from sklearn.metrics import mean_squared_error

In [2]:
N = 100000
N_SUBGRP = N//2
N_GRP = 0 # we will use only one level of pooling in this tutorial
BETA_TRUE = [2,1,0,0,0]
SIGMA = 1

In [3]:
# main data frame with features X, subgroup IDs subgrp_id and group ID grp_id;
# target y is NOT observable on the individual level in real data,
# we have it here to be able to simulate target on group level
# and to be able to compared result of the estimate on the group-level
# target with the estimate on the individual level.
df_train = gen_random_dataset(N, N_SUBGRP, N_GRP, BETA_TRUE, SIGMA)
df_val = gen_random_dataset(N, N_SUBGRP, N_GRP, BETA_TRUE, SIGMA)

In [4]:
df_train.head()

,_row_,X,y,subgrp_id
0,0,"[1.4408917701523807, 0.3824008277699567, 0.977...",2.417896,SUBGRP0
1,1,"[0.3024580422979794, -0.7448060343255809, -1.1...",1.422314,SUBGRP1
2,2,"[-0.10864955453327864, 0.4068987712793117, -0....",0.319509,SUBGRP2
3,3,"[-0.011951238497885106, -1.275706951800336, 0....",-1.091057,SUBGRP3
4,4,"[-1.7340322197175946, 0.0725599434274694, -0.1...",-4.321461,SUBGRP4


In [5]:
X_train = np.asarray(df_train['X'].to_list())
y_train = np.asarray(df_train['y'].to_list())
X_val = np.asarray(df_val['X'].to_list())
y_val = np.asarray(df_val['y'].to_list())

Calculate simulated target `y` on the level of `subgrp_id` (by max pooling of individual-level `y`'s).

In [6]:
df_train_subgrp_y = (df_train
    .groupby('subgrp_id')
    .agg({'y':np.max})
    .reset_index()
)
df_train_subgrp_inds = get_ragged_nested_index_lists(df_train, ['subgrp_id'])

In [7]:
df_val_subgrp_y = (df_val
    .groupby('subgrp_id')
    .agg({'y':np.max})
    .reset_index()
)
df_val_subgrp_inds = get_ragged_nested_index_lists(df_val, ['subgrp_id'])

## Custom TF Pooling and Loss Function

In [8]:
@xgb_tf_loss(df_train_subgrp_inds.sort_values(by=['subgrp_id'])['_row_'].to_list(), 
             df_train_subgrp_y.sort_values(by=['subgrp_id'])['y'].to_numpy())
def max_pooling_mse_loss(target, preds_cube):
    """Custom TF Pooling and Loss function.

    This example function performs max pooling from the individual
    level to subgroups.
    The function takes appropriate care of missing values in preds_cube.

    Inputs:
    = target: 1D tensor with target on the level of groups
    = preds_cube: ND tensor with predictions on the individual level;
    the first dimension is that of groups, the other dimensions reflect
    sub-groups on different levels and individual observations
    (target.shape[0] == preds_cube.shape[0]; 
    preds_cube.shape[-1] == max # indiv observations per the most detailed 
    sub-group).
    Missing values are denoted by np.nan and have to be taken care of in 
    this function body. They occur simply because preds_cube
    has typically much more elements that the original flat predictions
    vector from XGBoost.

    Output: scalar tensor reflecting MEAN of losses over all dimensions.
    This is the output of e.g. tf.keras.losses.mean_squared_error().
    The mean is translated to SUM later in tf_d_loss() because of the 
    compatibility with XGB custom objective function.
    """
    x = preds_cube
    # replace NaNs with -Inf: neutral value for reduce_max()
    x = tf.where(tf.math.is_nan(x), tf.constant(-np.inf, dtype=x.dtype), x)
    x = tf.math.reduce_max(x, axis=-1)
    l = tf.keras.losses.mean_squared_error(target, x)
    return l

## Custom Pooling Metric

In [9]:
@xgb_tf_metric(df_val_subgrp_inds.sort_values(by=['subgrp_id'])['_row_'].to_list(), 
               df_val_subgrp_y.sort_values(by=['subgrp_id'])['y'].to_numpy())
def max_pooling_mse_metric(target, preds_cube):
    """Custom Pooling MSE.

    This example function performs max pooling from the individual
    level to subgroups and computes MSE.
    The function takes appropriate care of missing values in preds_cube.

    Inputs:
    = target: 1D tensor with target on the level of groups
    = preds_cube: ND tensor with predictions on the individual level;
    the first dimension is that of groups, the other dimensions reflect
    sub-groups on different levels and individual observations
    (target.shape[0] == preds_cube.shape[0]; 
    preds_cube.shape[-1] == max # indiv observations per the most detailed 
    sub-group).
    Missing values are denoted by np.nan and have to be taken care of in 
    this function body. They occur simply because preds_cube
    has typically much more elements that the original flat predictions
    vector from XGBoost.

    Output: tuple (metric_name, metric_value)
    """
    preds_cube = np.nan_to_num(preds_cube, nan=-np.inf)
    preds = np.max(preds_cube, axis=-1)
    score = mean_squared_error(target, preds)
    return 'max_mse', score

## Estimation

In [10]:
dtrain = xgb.DMatrix(X_train)
dval = xgb.DMatrix(X_val)

In [11]:
%%time
regr_subgrp = xgb.train({'tree_method': 'hist',
                         'seed': 1994,
                         'n_jobs': 20,
                         'learning_rate': 0.12,
                         'disable_default_eval_metric': 1
                        }, 
                        num_boost_round=100,
                        dtrain=dtrain,
                        evals=[(dval, 'val')],
                        obj=max_pooling_mse_loss,
                        feval=max_pooling_mse_metric,
                        early_stopping_rounds=3
                       )

[0]	val-max_mse:4.50253
Will train until val-max_mse hasn't improved in 3 rounds.
[1]	val-max_mse:3.72263
[2]	val-max_mse:3.11670
[3]	val-max_mse:2.64586
[4]	val-max_mse:2.27652
[5]	val-max_mse:1.98936
[6]	val-max_mse:1.76443
[7]	val-max_mse:1.58828
[8]	val-max_mse:1.45016
[9]	val-max_mse:1.34253
[10]	val-max_mse:1.25848
[11]	val-max_mse:1.19267
[12]	val-max_mse:1.14106
[13]	val-max_mse:1.10057
[14]	val-max_mse:1.06903
[15]	val-max_mse:1.04438
[16]	val-max_mse:1.02479
[17]	val-max_mse:1.00978
[18]	val-max_mse:0.99782
[19]	val-max_mse:0.98840
[20]	val-max_mse:0.98137
[21]	val-max_mse:0.97561
[22]	val-max_mse:0.97118
[23]	val-max_mse:0.96781
[24]	val-max_mse:0.96481
[25]	val-max_mse:0.96273
[26]	val-max_mse:0.96098
[27]	val-max_mse:0.95967
[28]	val-max_mse:0.95853
[29]	val-max_mse:0.95760
[30]	val-max_mse:0.95702
[31]	val-max_mse:0.95654
[32]	val-max_mse:0.95606
[33]	val-max_mse:0.95578
[34]	val-max_mse:0.95551
[35]	val-max_mse:0.95538
[36]	val-max_mse:0.95529
[37]	val-max_mse:0.95513
[3